In [3]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8")

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

## Part 1: Accuracy  

**Variables**

**TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

**TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> this is a analyst forecast variable

**TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> forecast error between actual EPS and TR.EPSMean  


In [4]:
#accuracy_variables = ['TR.RevenueActValue.date', 'TR.RevenueActValue', "TR.F.EV", 'TR.EPSActValue', "TR.EPSMean", "TR.EpsSmartEst", "TR.EPSActSurprise", "TR.EpsPreSurprisePct"]
accuracy_variables = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise"]

In [5]:
df_accuracy, e = ek.get_data('0#.SPX',accuracy_variables, parameters = {'SDate':'0','EDate':'-5','Period':'FQ0','Frq':'FQ'})
df_accuracy["Date"] = pd.to_datetime(df_accuracy["Date"])

In [6]:
df_accuracy

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise
0,POOL.OQ,2023-02-16 07:00:00+00:00,1.82,1.987,-8.405
1,POOL.OQ,2022-10-20 07:00:00+00:00,4.78,4.5875,4.196
2,POOL.OQ,2022-07-21 07:00:00+00:00,7.63,7.517,1.503
3,POOL.OQ,2022-04-21 07:00:00+00:00,4.23,3.14867,34.342
4,POOL.OQ,2022-02-17 07:00:00+00:00,2.63,1.875,40.267
...,...,...,...,...,...
3003,AVY.N,2022-10-26 06:45:00+00:00,2.46,2.46323,-0.131
3004,AVY.N,2022-07-27 06:45:00+00:00,2.64,2.36485,11.635
3005,AVY.N,2022-04-26 06:45:00+00:00,2.4,2.17008,10.595
3006,AVY.N,2022-02-02 06:45:00+00:00,2.13,2.12375,0.294


In [7]:
df_accuracy.dtypes

Instrument                                           string
Date                                    datetime64[ns, UTC]
Earnings Per Share - Actual                         Float64
Earnings Per Share - Mean                           Float64
Earnings Per Share - Actual Surprise                Float64
dtype: object

### Exploratory Data Analysis for Analyst Forcast Accuracy

Mean for the entire time frame for each instrument (just to get an overview)

In [8]:
df_averages = df_accuracy.groupby("Instrument").mean()
df_averages

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_29861/3444857495.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_averages = df_accuracy.groupby("Instrument").mean()


,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise
Instrument,,,
A.N,1.253333,1.17476,6.625667
AAL.OQ,-0.351667,-0.408517,6.156333
AAP.N,3.138333,3.095295,2.1005
AAPL.OQ,1.538333,1.486917,3.306167
ABBV.N,3.418333,3.349458,2.076
...,...,...,...
YUM.N,1.123333,1.12382,-0.0935
ZBH.N,1.775,1.694997,5.18
ZBRA.OQ,4.43,4.276117,3.820833


categorizing the forecast accuracy:

## Part 2: Factors

### Analysts
**TR.NumberOfAnalysts** - Number of sell-side analysts covering the security.

**TR.RecMean** - Recommendation Numeric Mean based on the Standard Scale Of Strong Buy (1 ), Buy (2), Hold (3), Sell (4), and Strong Sell (5).

### continue..

**TR.WACCInflationAdjRiskFreeRate** -  Inflation adjusted risk free rate used is Of the primary index for the domicile country Of the company. Inflation adjusted risk free rate (Rf) is calculated from the US 10-year treasury yield plus the difference between in the 10-year forecasted inflation rate between the given country and the US.

## Part 3: Regression

### continue..

In [39]:
economic_story = ['TR.RevenueActValue.date', 'TR.RevenueActValue', "TR.F.EV", 'TR.EPSActValue', "TR.EPSMean", "TR.EpsSmartEst", "TR.EPSActSurprise", "TR.NumberOfAnalysts", "TR.RecMean", "TR.WACCInflationAdjRiskFreeRate"]
#["TR.TRBCEconomicSector", "TR.F.EV", "TR.EPSActValue", "TR.NumberOfAnalysts", "TR.NumOfRecommendations", "TR.EpsPreSurprisePct", "TR.EPSActSurprise", "TR.RecMean", "TR.TRESGManagementScore"]

In [40]:
df, e = ek.get_data('0#.SPX',economic_story, parameters = {'SDate':'0','EDate':'-5','Period':'FQ0','Frq':'FQ'})
df["Date"] = pd.to_datetime(df["Date"])

C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1134: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\pandas\core\dtypes\cast.py:1153: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


In [41]:
df

,Instrument,Date,Revenue - Actual,Enterprise Value,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - SmartEstimate®,Earnings Per Share - Actual Surprise,Number of Analysts,Recommendation - Mean (1-5),"WACC Inflation Adjusted Risk Free Rate, (%)"
0,POOL.OQ,2022-10-20 07:00:00+00:00,1615339000.0,14061600372.299999,4.78,4.56,4.55802,4.825,10,2.27273,3.831224
1,POOL.OQ,2022-07-21 07:00:00+00:00,2055818000.0,15561870705.290001,7.63,7.517,7.54885,1.503,9,2.2,3.747421
2,POOL.OQ,2022-04-21 07:00:00+00:00,1412650000.0,18436823034.200001,4.23,3.14867,3.15428,34.342,9,2.0,3.092855
3,POOL.OQ,2022-02-17 07:00:00+00:00,1035557000.0,23848820586.0,2.63,1.875,1.88052,40.267,8,2.3,2.325202
4,POOL.OQ,2021-10-21 07:00:00+00:00,1411448000.0,17699543822.66,4.51,3.84833,3.86379,17.194,8,2.3,1.515266
...,...,...,...,...,...,...,...,...,...,...,...
3013,AVY.N,2022-10-26 06:45:00+00:00,2317100000.0,16225017418.9,2.46,2.46323,2.46223,-0.131,13,2.14286,3.747421
3014,AVY.N,2022-07-27 06:45:00+00:00,2347000000.0,16504297189.84,2.64,2.36485,2.38256,11.635,13,2.0,3.092855
3015,AVY.N,2022-04-26 06:45:00+00:00,2349300000.0,17537077488.32,2.4,2.17008,2.16901,10.595,12,2.06667,2.325202
3016,AVY.N,2022-02-02 06:45:00+00:00,2183200000.0,20873039410.310001,2.13,2.12375,2.12724,0.294,12,2.14286,1.515266


In [42]:
4.23/3.14867

1.343424366478545

In [43]:
df.dtypes

Instrument                                                  string
Date                                           datetime64[ns, UTC]
Revenue - Actual                                           Float64
Enterprise Value                                           Float64
Earnings Per Share - Actual                                Float64
Earnings Per Share - Mean                                  Float64
Earnings Per Share - SmartEstimate®                        Float64
Earnings Per Share - Actual Surprise                       Float64
Number of Analysts                                           Int64
Recommendation - Mean (1-5)                                Float64
WACC Inflation Adjusted Risk Free Rate, (%)                Float64
dtype: object